### 데이터베이스 테스트

In [ ]:
"""
<DB 접속 및 사용 순서>
1. DB 드라이버 연결 settings.py
2. DB Connection 접속 settings.py
3. 커서 받아오기 models.py
4. SQL 구문 작성 및 실행요청(커서가 처리함) views.py
5. 결과 출력을 위한 프로그래밍 templates
6. 커서 반환 > DB Connection 접속 해제 settings.py
"""

In [46]:
### 오라클 라이브러리 불러들이기
import cx_Oracle

### 1. DB 드라이버 연결하기(서버 찾아가기)

In [47]:
dsn = cx_Oracle.makedsn("localhost",1521,"xe")
# django에서 settings.py에 DATABASES에 오라클 데이터베이스 설정 추가에 있음

### 2. DB 접속(connect)하기(통로 만들기)

In [48]:
conn = cx_Oracle.connect("gwangju_a","dbdb",dsn) 
# sqldeveloper에서 접속정보의 사용자 이름과 비밀번호가 들어감

### 3. 커서(cusor)받아오기(서버로 sql구문을 실어나르고, 결과 받아오는 역할)

In [49]:
cursor = conn.cursor()

### 4. SQL 구문 작성 및 실행요청(커서가 처리함)

In [50]:
### SQL 요청 구문 작성하기
# 회원정보에서 회원아이디, 회원이름, 회원주소1 조회하기
# sqldeveloper에서 실행한 sql 구문과 동일하게 넣어줘야함
sql = """
    select mem_id, mem_name, mem_add1
    From member
    order by mem_name
"""

In [51]:
### SQL 실행요청
# 구문을 서버에게 보내서 요청하고, 결과를 받아오기 -> 커서가 수행함
cursor.execute(sql)

<cx_Oracle.Cursor on <cx_Oracle.Connection to gwangju_a@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>>

### 5. 결과 출력을 위한 프로그래밍

In [52]:
### 커서가 DB서버로부터 받아온 결과값 추출하기
# - 여러건 조회 : fetchall()
# - 한건 조회   : fetchone()
# 이 데이터를 웹에서 사용하려면 딕셔너리형태로 바꿔야함
rows = cursor.fetchall()

In [53]:
rows

[('l001', '구길동', '충남금산군 금산읍'),
 ('u001', '김성욱', '대전시 동구 용전동'),
 ('j001', '김윤희', '대전시 서구 삼천동'),
 ('a001', '김은대', '대전시 동구 용운동'),
 ('w001', '김형모', '대전시 대덕구 연축동'),
 ('h001', '라준호', '충남 논산시 양촌면'),
 ('m001', '박지은', '대전광역시 서구 갈마동'),
 ('o001', '배인정', '대전시 서구 갈마동'),
 ('t001', '성원태', '대전광역시 중구 유천동'),
 ('d001', '성윤미', '대전시 중구 하늘동 '),
 ('g001', '송경희', '충남금산군 제원면'),
 ('f001', '신영남', '대전광역시 대흥동'),
 ('c001', '신용환', '대전광역시 중구 대흥동'),
 ('s001', '안은정', '대구광역시 서구 탄방동'),
 ('p001', '오성순', '대전유성구송강동'),
 ('k001', '오철희', '대전시 대덕구 대화동'),
 ('q001', '육평회', '대구광역시 대덕구 중리동'),
 ('b001', '이쁜이', '서울시 천사동 예쁜마을'),
 ('v001', '이진영', '대전시 동구 용전동'),
 ('e001', '이혜나', '대전시 대덕구 읍내동'),
 ('r001', '정은실', '대전시 동구 용전동'),
 ('x001', '진현경', '대전광역시 동구 오정동'),
 ('i001', '최지현', '대전시 동구 가양1동'),
 ('n001', '탁원재', '대전시 동구 자양동')]

In [55]:
### 조회결과에 대한 컬럼명만 조회하기 (커서가 가지고 있음)
colname = cursor.description

In [67]:
# 데이터베이스에서 조회한 결과의 컬럼명은 모두 대문자
# - 프로그램에서 사용할 때는 소문자로 바꿔서 사용!
colname1=colname[0][0].lower()
colname2=colname[1][0].lower()
colname3=colname[2][0].lower()

In [69]:
col = [colname1,colname2,colname3]
col

['mem_id', 'mem_name', 'mem_add1']

In [74]:
### 컬럼명만 추출하여 col = [] 변수에 담기
# - 추출한 컬럼명은 소문자로 변환해서 col 리스트 변수에 담기

# for문 안에서 값을 담아서 누적 시키기 위해 전역변수로 선언
col = []
for t in colname :
    # col 리스트 변수에 값을 추가하기 : append(값) 사용
    col.append(t[0].lower())
col

['mem_id', 'mem_name', 'mem_add1']

In [94]:
### html에서 사용할 수 있도록 하기 위해서
# - [{"mem_id": "a001", "mem_name" : "asdfasdf", "mem_add1" : "광주"},
#   {"mem_id": "a002", "mem_name" : "asdfasdf", "mem_add2" : "광주"},
#   {},{}.....] 형태로 변환하기
# - 최종 결과값을 담을 변수면 : list_row = []
list_row = []
for t in rows:
    dict_temp = {}
    for i in range(0, len(col), 1):
#         print(col[i], t[i])
        dict_temp[col[i]] = t[i]
    list_row.append(dict_temp)
list_row

[{'mem_id': 'l001', 'mem_name': '구길동', 'mem_add1': '충남금산군 금산읍'},
 {'mem_id': 'u001', 'mem_name': '김성욱', 'mem_add1': '대전시 동구 용전동'},
 {'mem_id': 'j001', 'mem_name': '김윤희', 'mem_add1': '대전시 서구 삼천동'},
 {'mem_id': 'a001', 'mem_name': '김은대', 'mem_add1': '대전시 동구 용운동'},
 {'mem_id': 'w001', 'mem_name': '김형모', 'mem_add1': '대전시 대덕구 연축동'},
 {'mem_id': 'h001', 'mem_name': '라준호', 'mem_add1': '충남 논산시 양촌면'},
 {'mem_id': 'm001', 'mem_name': '박지은', 'mem_add1': '대전광역시 서구 갈마동'},
 {'mem_id': 'o001', 'mem_name': '배인정', 'mem_add1': '대전시 서구 갈마동'},
 {'mem_id': 't001', 'mem_name': '성원태', 'mem_add1': '대전광역시 중구 유천동'},
 {'mem_id': 'd001', 'mem_name': '성윤미', 'mem_add1': '대전시 중구 하늘동 '},
 {'mem_id': 'g001', 'mem_name': '송경희', 'mem_add1': '충남금산군 제원면'},
 {'mem_id': 'f001', 'mem_name': '신영남', 'mem_add1': '대전광역시 대흥동'},
 {'mem_id': 'c001', 'mem_name': '신용환', 'mem_add1': '대전광역시 중구 대흥동'},
 {'mem_id': 's001', 'mem_name': '안은정', 'mem_add1': '대구광역시 서구 탄방동'},
 {'mem_id': 'p001', 'mem_name': '오성순', 'mem_add1': '대전유성구송강동'},
 {'me

In [98]:
### 회원정보에서 회원아이디가 'a001'에 대한 회원아이디, 회원이름, 회원주소1 조회하기
# --> 한건 조회 --> 튜플형태로 가져와줌
sql = """
    Select mem_id, mem_name, mem_add1
    From member
    Where mem_id = 'a001'
"""

In [99]:
cursor.execute(sql)

<cx_Oracle.Cursor on <cx_Oracle.Connection to gwangju_a@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>>

In [101]:
row = cursor.fetchone()

In [102]:
row

('a001', '김은대', '대전시 동구 용운동')

In [103]:
colname = cursor.description

In [104]:
colname

[('MEM_ID', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 15, 15, None, None, 0),
 ('MEM_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 20, 20, None, None, 0),
 ('MEM_ADD1', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 100, 100, None, None, 0)]

In [113]:
### 한건 조회 결과를 딕셔너리 형태로 변환하기
# 최종 변수이름 : dict_row

### 컬럼명만 조회
col = []
for t in colname:
    # col 리스트 변수에 값을 추가하기 : append(값) 사용
    col.append(t[0].lower())
col

['mem_id', 'mem_name', 'mem_add1']

In [114]:
### 딕셔너리로 변환
dict_row = {}
for i in range(0, len(col),1):
    dict_row[coln[i]]= row[i]
dict_row

{'mem_id': 'a001', 'mem_name': '김은대', 'mem_add1': '대전시 동구 용운동'}

### 함수로 기능 정의하기

### - DB접속 기능 함수 정의

In [115]:
def getDBConn_Cursor() :
    dsn = cx_Oracle.makedsn("localhost",1521,"xe")
    conn = cx_Oracle.connect("gwangju_a","dbdb",dsn) 
    cursor = conn.cursor()
    return conn, cursor

##### - 조회결과에서 컬럼명 추출하기 기능 정의

In [116]:
def getColName(cursor):
    col = []
    for t in cursor.description :
        col.append(t[0].lower())
    return col

##### - "한건 조회"시 딕셔너리로 변환하는 기능 정의

In [128]:
def getFetchOne(cursor, row):
    # 컬럼명 조회 함수 호출
    col = getColName(cursor)
    
    dict_row = {}
    for i in range(0, len(col),1):
        dict_row[col[i]]= row[i]
        
    return dict_row

##### - "여러건 조회"시 리스트의 딕셔너리로 변환하는 기능 정의

In [129]:
def getFetchAll(cursor, rows):
     # 컬럼명 조회 함수 호출
    col = getColName(cursor)
    
    list_row = []
    for t in rows:
        dict_temp = {}
        for i in range(0, len(col), 1):
            dict_temp[col[i]] = t[i]
        list_row.append(dict_temp)
        
    return list_row

##### - 커서 및 connect 접속 해제하는 기능 정의

In [130]:
def DBClose(cursor, conn):
    ### 커서(cursor)반환하기
    cursor.close()
    
    ### 접속(connect) 접속 해제
    conn.close()

### 6. 커서 반환 > DB Connection 접속 해제

In [131]:
### 커서(cursor)반환하기
cursor.close()
### 접속(connect) 접속 해제
conn.close()

### 함수 호출 방식으로 프로그래밍하기

In [132]:
### DB 접속 정보 받아오기
conn, cursor = getDBConn_Cursor()

### 구문 작성
"""
    select mem_id, mem_name, mem_add1
    From member
    Where mem_id = 'a001'
"""
### 구문 실행하기
cursor.execute(sql)

### 결과 추출하기
row = cursor.fetchone()

### 딕셔너리로 변경하기
dict_row = getFetchOne(cursor, row)
print(dict_row)

### DB 접속 해제하기
DBClose(cursor, conn)

{'mem_id': 'l001', 'mem_name': '구길동', 'mem_add1': '충남금산군 금산읍'}
